# AR6-based forcing. Part 2

Based on `12_make-forcing.py` and `13_radiative-forcing-barchart.py`
in https://github.com/ClimateIndicator/forcing-timeseries/notebooks

In [1]:
import numpy as np
import pandas as pd
import h5py
from scipy.stats import linregress
from mce import MCEExecError
from mce.core import ScenarioBase
from mce.core.forcing_ar6 import RfAllAR6
from mce.util.io import RetrieveGitHub

## Reference data

In [2]:
repo = RetrieveGitHub('ClimateIndicator', 'forcing-timeseries', 'datain')

In [3]:
dfref = {}
path = repo.retrieve('output/ERF_best_1750-2024.csv')
dfref['erf_best'] = pd.read_csv(path, index_col=0)
path = repo.retrieve('output/ERF_best_aggregates_1750-2024.csv')
dfref['erf_best_agg'] = pd.read_csv(path, index_col=0).rename(int)
path = repo.retrieve('output/ERF_p05_aggregates_1750-2024.csv')
dfref['erf_p05_agg'] = pd.read_csv(path, index_col=0).rename(int)
path = repo.retrieve('output/ERF_p95_aggregates_1750-2024.csv')
dfref['erf_p95_agg'] = pd.read_csv(path, index_col=0).rename(int)

[2025-03-18 10:57:33 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/output/ERF_best_1750-2024.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/raw/main/output/ERF_best_1750-2024.csv on 2025-03-11
[2025-03-18 10:57:33 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/output/ERF_best_aggregates_1750-2024.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/raw/main/output/ERF_best_aggregates_1750-2024.csv on 2025-03-11
[2025-03-18 10:57:33 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/output/ERF_p05_aggregates_1750-2024.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/raw/main/output/ERF_p05_aggregates_1750-2024.csv on 2025-03-11
[2025-03-18 10:57:33 mce.util.io] INFO:Use local file datain/ClimateIndicator/forcing-timeseries/output/ERF_p95_aggregates_1750-2024.csv retrieved from https://github.com/ClimateIndicator/forcing-timeseries/r

## Input data

### Indicators of Global Climate Change

In [4]:
h5f = h5py.File('untracked/datain.h5', 'r')
dgrp = h5f['ClimateIndicator-2024/forcing-timeseries']

In [5]:
def func(name, obj):
    desc = obj.attrs.get('description')
    if desc is not None:
        print('{}: {}'.format(name, desc))

dgrp.visititems(func)

aci_cal: Forcing uncertainty ensemble related to aerosol-cloud interactions
ari_emitted: Reference radiative forcing by SLCF species
skeie_ozone_strat: Reference radiative forcing of stratospheric ozone
skeie_ozone_trop: Reference radiative forcing of tropospheric ozone
timeseries/erf_contrails: Effective radiative forcing of contrails and contrail-induced cirrus from 1930 to 2024
timeseries/erf_irrigation: Land use forcing due to irrigation
timeseries/erf_solar: Effective radiative forcing of solar irradiance from -6755 to 2299
timeseries/erf_volcanic: Effective radiative forcing of volcanic activity from -9499 to 2024
timeseries/gcp_emissions: Emissions of CO2 from 1750 to 2024
timeseries/ghg_concentrations: Concentrations of GHGs from 1750 to 2024
timeseries/sarf_landuse: Land use forcing due to albedo change
timeseries/slcf_emissions: Emissions of SLCF species from 1750 to 2024
timeseries/temp_obs: Temperature time series used for temperature feedback of ozone forcing from 1850 to 

### Construct input timeseries

In [6]:
class Scenario(ScenarioBase):
    def init_process(self, name, dgrp_in, **kw):
        year = np.arange(1750, 2024+1)

        # GHG concentrations
        gin = dgrp_in['timeseries/ghg_concentrations']
        gout = self.file.create_group(f'{name}/input/conc')
        dset = gout.create_dataset('time', data=year)
        dset.attrs['units'] = 'yr'
        df = pd.DataFrame({
            k: v[:] for k, v in gin.items()
        }).set_index('year').rename_axis(None)
        df = df.reindex(year).interpolate()
        for k, v in df.items():
            dset = gout.create_dataset(k, data=v.values)
            dset.attrs['units'] = gin[k].attrs['units']

        # CO2 emissions
        gin = dgrp_in['timeseries/gcp_emissions']
        if not np.array_equal(year, gin['year'][:]):
            raise MCEExecError('inconsistent year')

        gout = self.file.create_group(f'{name}/input/emis_co2')
        dset = gout.create_dataset('time', data=year)
        dset.attrs['units'] = 'yr'
        for k, v in gin.items():
            if k == 'year':
                continue

            dset = gout.create_dataset(k, data=v[:])
            dset.attrs['units'] = v.attrs['units']

        # SLCF emissions
        gin = dgrp_in['timeseries/slcf_emissions']
        if not np.array_equal(year, gin['year'][:]):
            raise MCEExecError('inconsistent year')

        gout = self.file.create_group(f'{name}/input/emis_slcf')
        dset = gout.create_dataset('time', data=year)
        dset.attrs['units'] = 'yr'
        for k, v in gin.items():
            if k == 'year':
                continue

            dset = gout.create_dataset(k, data=v[:])
            dset.attrs['units'] = v.attrs['units']

        # ERF of land_use, contrails, solar, and volcanics
        gout = self.file.create_group(f'{name}/input/erf_other')
        dset = gout.create_dataset('time', data=year)
        dset.attrs['units'] = 'yr'

        gin = dgrp_in['timeseries/sarf_landuse']
        d1 = pd.Series(gin['LUH2-GCB2024'][:], index=gin['year'][:])
        d1 *= -0.15 / d1.loc[2004]
        d1.loc[2024] = d1.loc[2023]
        df = d1.to_frame('land_use')

        gin = dgrp_in['timeseries/erf_irrigation']
        d1 = pd.Series(gin['value'][:], index=gin['year'][:])
        lr = linregress(d1.loc[2013:].index, d1.loc[2013:])
        d1.loc[2023] = lr.slope * 2023 + lr.intercept
        d1.loc[2024] = lr.slope * 2024 + lr.intercept
        df['irrigation'] = d1.reindex(df.index)

        for k, v in df.items():
            dset = gout.create_dataset(k, data=v.loc[1750:].values)
            dset.attrs['units'] = 'W m-2'

        k = 'contrails'
        gin = dgrp_in[f'timeseries/erf_{k}']
        d1 = pd.Series(gin['value'][:], index=gin['year'][:])
        d1 = d1.reindex(year, fill_value=0.)
        dset = gout.create_dataset(k, data=d1.values)
        dset.attrs['units'] = 'W m-2'

        for k in ['solar', 'volcanic']:
            gin = dgrp_in[f'timeseries/erf_{k}']
            d1 = pd.Series(gin['value'][:], index=gin['year'][:])
            d1 = d1.loc[year[0]:year[-1]]
            if not np.array_equal(year, d1.index.values):
                raise MCEExecError('inconsistent year')

            dset = gout.create_dataset(k, data=d1.values)
            dset.attrs['units'] = 'W m-2'

In [7]:
ds = Scenario('historical', dgrp, outpath='untracked/scenarios.h5', mode='a')

[2025-03-18 10:58:00 mce.core] INFO:untracked/scenarios.h5 already exists
[2025-03-18 10:58:00 mce.core] INFO:file untracked/scenarios.h5 opened with mode=a


In [8]:
ds_hist = ds.get_scenario('historical')
list(ds_hist)

['conc', 'emis_co2', 'emis_slcf', 'erf_other']

In [9]:
ds.close()

[2025-03-18 10:58:13 mce.core] INFO:file untracked/scenarios.h5 closed


In [10]:
dcat = ds_hist['emis_slcf']
df_emis_slcf = pd.DataFrame(
    dcat['data'], index=dcat['time'], columns=dcat['variables'],
)
dcat = ds_hist['conc']
df_conc = pd.DataFrame(
    dcat['data'], index=dcat['time'], columns=dcat['variables'],
)

In [11]:
df_scale = pd.DataFrame({
    k: v[:]
    for k, v in dgrp['unc']['scale'].items()
})
trend_solar = pd.Series(dgrp['unc/trend_solar'][:])

### Construct forcing instance

In [12]:
forcing = RfAllAR6(df_emis_slcf, df_conc)

In [13]:
gin = dgrp['ari_emitted']
df = pd.DataFrame(
    {k: v[:] for k, v in gin.items()},
    index=gin.attrs['species'],
)
forcing.init__ari(df['mean'], df['sd'])

gin = dgrp['aci_cal']
df = pd.DataFrame(
    {k: v[:] for k, v in gin.items()},
    index=gin.attrs['models'],
)
forcing.init__aci(df)

gin = dgrp['timeseries/temp_obs']
args = [pd.Series(gin['value'][:], index=gin['year'][:])]

for k in ['trop', 'strat']:
    gin = dgrp[f'skeie_ozone_{k}']
    args.append(
        pd.DataFrame(
            gin['value'][:],
            index=gin.attrs['index_model'],
            columns=gin.attrs['columns_year'],
        )
    )

forcing.init__o3(*args)

In [14]:
h5f.close()

## Make ERF timeseries

### Best estimate

In [15]:
df = forcing.erf__ghg_minor(df_conc)
ghgs_minor = df.columns.tolist()

erf_best = pd.concat([
    forcing.erf__ghg_major(df_conc),
    df,
], axis=1)

In [16]:
cat = 'aerosol-radiation_interactions'
erf_best[cat] = forcing.erf__ari(df_emis_slcf, df_conc)

In [17]:
cat = 'aerosol-cloud_interactions'
erf_best[cat] = forcing.erf__aci(df_emis_slcf)

In [18]:
cat = 'O3'
erf_best[cat] = forcing.erf__o3(df_emis_slcf, df_conc)

In [19]:
cat = 'BC_on_snow'
d1 = df_emis_slcf['BC']
erf_best[cat] = forcing.bc_on_snow__factor * (d1 - d1.loc[1750])

In [20]:
cat = 'H2O_stratospheric'
d1 = df_conc['CH4']
erf_best[cat] = forcing.h2o_strat__factor * (d1 - d1.loc[1750])

In [21]:
din = ds_hist['erf_other']
df = pd.DataFrame(din['data'], index=din['time'], columns=din['variables'])

cats = ['irrigation', 'land_use']
erf_best = pd.concat([
    erf_best,
    df.drop(cats, axis=1),
    df[cats].sum(axis=1).to_frame('land_use'),
], axis=1)

df__erf_other = df

In [22]:
erf_best.shape, dfref['erf_best'].shape

((275, 61), (275, 61))

In [23]:
np.allclose(
    erf_best,
    dfref['erf_best'][erf_best.columns],
)

True

In [24]:
erf_best_agg = erf_best.drop(ghgs_minor, axis=1)

In [25]:
cats = ['aerosol-radiation_interactions', 'aerosol-cloud_interactions']
erf_best_agg['aerosol'] = erf_best[cats].sum(axis=1)

erf_best_agg['halogen'] = erf_best[ghgs_minor].sum(axis=1)
erf_best_agg['nonco2wmghg'] = erf_best[['CH4', 'N2O'] + ghgs_minor].sum(axis=1)

cats = ['H2O_stratospheric', 'contrails', 'BC_on_snow', 'land_use']
erf_best_agg['minor'] = erf_best[cats].sum(axis=1)

cats = ['solar', 'volcanic']
erf_best_agg['anthro'] = erf_best.drop(cats, axis=1).sum(axis=1)

erf_best_agg['total'] = erf_best.sum(axis=1)

In [26]:
erf_best_agg.shape, dfref['erf_best_agg'].shape

((275, 18), (275, 18))

In [27]:
np.allclose(
    erf_best_agg,
    dfref['erf_best_agg'][erf_best_agg.columns],
)

True

### 90% uncertainty range

In [28]:
dfref_unc = pd.concat({
    'p05': dfref['erf_p05_agg'],
    'p95': dfref['erf_p95_agg'],
}, axis=1).reorder_levels([1, 0], axis=1).sort_index(axis=1)

In [29]:
cats_agg = list(erf_best_agg)

In [30]:
[k for k in cats_agg if k not in erf_best]

['aerosol', 'halogen', 'nonco2wmghg', 'minor', 'anthro', 'total']

In [31]:
ens_agg = {
    k: 0. for k in [
        'aerosol',
        'nonco2wmghg',
        'minor',
        'anthro',
        'total',
    ]
}
ens_agg_count = {
    k: [] for k in [
        'aerosol',
        'nonco2wmghg',
        'minor',
        'anthro',
        'total',
    ]
}
ens_pct = {}

In [32]:
cat = 'aerosol-radiation_interactions'
dfin = forcing.save__ari_dfin
radeff_ens = forcing.ari__radeff_ens[dfin.keys()]
ens = (
    dfin.values[:, None, :] * radeff_ens.values[None, :, :]
).sum(axis=2)
ens_pct[cat] = np.percentile(ens, [5, 95], axis=1).T

for k in ['aerosol', 'anthro', 'total']:
    ens_agg[k] += ens
    ens_agg_count[k].append(cat)

In [33]:
cat = 'aerosol-cloud_interactions'
ens = forcing.save__aci_ens.values
ens_pct[cat] = np.percentile(ens, [5, 95], axis=1).T

for k in ['aerosol', 'anthro', 'total']:
    ens_agg[k] += ens
    ens_agg_count[k].append(cat)

In [34]:
cat = 'land_use'
ens = (
    df__erf_other['irrigation'].values[:, None] * df_scale['irrigation'].values
    + df__erf_other['land_use'].values[:, None] * df_scale['land_use'].values
)
ens_pct[cat] = np.percentile(ens, [5, 95], axis=1).T

for k in ['minor', 'anthro', 'total']:
    ens_agg[k] += ens
    ens_agg_count[k].append(cat)

In [35]:
cat = 'solar'
d1 = pd.Series(np.nan, index=erf_best.index)
d1.loc[1750] = 0.
d1.loc[2019:] = 1.
d1 = d1.interpolate()
ens = (
    erf_best[cat].values[:, None] * df_scale[cat].values
    + d1.values[:, None] * trend_solar.values[None, :]
)
ens_pct[cat] = np.percentile(ens, [5, 95], axis=1).T

ens_agg['total'] += ens
ens_agg_count['total'].append(cat)

In [36]:
cat = 'halogen'
ens = 0.
for k in ghgs_minor:
    ens += erf_best[k].values[:, None] * df_scale[k].values

ens_pct[cat] = np.percentile(ens, [5, 95], axis=1).T

for k in [
    'nonco2wmghg',
    'anthro',
    'total',
]:
    ens_agg[k] += ens
    ens_agg_count[k].append(cat)

In [37]:
for cat in cats_agg:
    if cat in ens_pct or cat in ens_agg:
        continue

    ens = erf_best[cat].values[:, None] * df_scale[cat].values
    ens_pct[cat] = np.percentile(ens, [5, 95], axis=1).T

    ens_agg['total'] += ens
    ens_agg_count['total'].append(cat)
    if cat not in ['solar', 'volcanic']:
        ens_agg['anthro'] += ens
        ens_agg_count['anthro'].append(cat)
    if cat in ['CH4', 'N2O']:
        ens_agg['nonco2wmghg'] += ens
        ens_agg_count['nonco2wmghg'].append(cat)
        # 'halogen' counted already
    if cat in ['H2O_stratospheric', 'contrails', 'BC_on_snow']:
        # 'land_use' counted already
        ens_agg['minor'] += ens
        ens_agg_count['minor'].append(cat)

In [38]:
for cat, ens in ens_agg.items():
    ens_pct[cat] = np.percentile(ens, [5, 95], axis=1).T

In [39]:
for i, (cat, pct) in enumerate(ens_pct.items()):
    print(i, np.allclose(pct, dfref_unc[cat]), cat)

0 True aerosol-radiation_interactions
1 True aerosol-cloud_interactions
2 True land_use
3 True solar
4 True halogen
5 True CO2
6 True CH4
7 True N2O
8 True O3
9 True BC_on_snow
10 True H2O_stratospheric
11 True contrails
12 True volcanic
13 True aerosol
14 True nonco2wmghg
15 True minor
16 True anthro
17 True total


In [40]:
ens_agg_count

{'aerosol': ['aerosol-radiation_interactions', 'aerosol-cloud_interactions'],
 'nonco2wmghg': ['halogen', 'CH4', 'N2O'],
 'minor': ['land_use', 'BC_on_snow', 'H2O_stratospheric', 'contrails'],
 'anthro': ['aerosol-radiation_interactions',
  'aerosol-cloud_interactions',
  'land_use',
  'halogen',
  'CO2',
  'CH4',
  'N2O',
  'O3',
  'BC_on_snow',
  'H2O_stratospheric',
  'contrails'],
 'total': ['aerosol-radiation_interactions',
  'aerosol-cloud_interactions',
  'land_use',
  'solar',
  'halogen',
  'CO2',
  'CH4',
  'N2O',
  'O3',
  'BC_on_snow',
  'H2O_stratospheric',
  'contrails',
  'volcanic']}